In [1]:
import pandas as pd
df = pd.read_csv('results_1008.csv')

In [19]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from scipy.spatial.distance import cosine

# Load pre-trained model tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Function to compute sentence embeddings
def embed_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

# Function to compute cosine similarity
def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

df['embedding'] = df['ai_metaphor'].apply(embed_sentence)

# def remove_similar_sentences(df, threshold=0.8):
embeddings = df['embedding'].tolist()

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [18]:
np.save('metaphor_embs.npy',embeddings)


# Find duplicates with ai-generated responses

In [9]:
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

# Initialize the pre-trained sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

gen_df = pd.read_csv('responses.csv') #ai generated responses

embeddings_df2 = model.encode(gen_df['response'].tolist(), convert_to_tensor=True)

# Compute cosine similarity between all pairs of sentences (df1 x df2)
cosine_sim = cosine_similarity(embeddings, embeddings_df2)

# Set the similarity threshold
threshold = 0.85

# For each sentence in df2, find sentences in df1 with cosine similarity > 0.9
similarity_results = []
too_similar = []
for idx, sentence in enumerate(gen_df['response']):
    # Get sentences in df1 with similarity > threshold
    similar_indices = np.where((cosine_sim[:, idx] > threshold))[0] #&(cosine_sim[:, idx] < 0.87)
    similar_sentences = df.ai_metaphor.iloc[similar_indices].tolist()
    too_similar.extend(list(similar_indices))
    # Store the result
    similarity_results.append(similar_sentences)
# too_similar = 
# Display the results
gen_df['sim']= similarity_results

In [10]:
filtered_df = gen_df[gen_df['sim'].apply(lambda x: len(x) > 0)]


In [11]:
for i,x in filtered_df.iterrows():
    print('GPT: '+x.response)
    print('Dataset: '+x.sim[0])

GPT: AI is a double-edged sword, like a genie in a bottle - incredibly powerful and capable of fulfilling our wishes, but also with the potential to unleash unforeseen consequences when set loose.
Dataset: AI is like building a robot that you know will destroy the world, but deciding to do it anyway, because it will make your like a little easier before the world ends.
GPT: AI is like a silent orchestra conductor, harmonizing the melodies of data and algorithms to create a symphony of insights.
Dataset: AI is like a cosmic symphony, its algorithms echoing celestial harmonies. It's an ethereal maestro orchestrating data's celestial dance, transforming raw notes into intricate melodies of insight. With each note played, AI unravels the universe's secrets, painting a portrait of knowledge, a cosmic symphony of intelligence.
GPT: AI is a brain without a body, perceiving the world through lines of code.
Dataset: AI is like a brain with no body
GPT: AI is a double-edged sword, like a genie i

In [13]:
len(set(too_similar))

276

In [14]:
too_sim = sorted(set(too_similar))

In [15]:
ai_generated = []
for i,x in df.iterrows():
    if i in too_sim:
        ai_generated.append(True)
    else:
        ai_generated.append(False)
df['ai_generated'] = ai_generated

In [16]:

# Specify the column to exclude
column_to_exclude = 'embedding'

# Create a new DataFrame excluding the specified column
df_excluded = df.drop(columns=[column_to_exclude,'Unnamed: 0.1'])

# Save the resulting DataFrame to a CSV file
df_excluded.to_csv('ai_metaphors_1010_checked_for_ai.csv',index=False)

In [17]:
df.ai_generated.value_counts()

ai_generated
False    12654
True       276
Name: count, dtype: int64